# Notebook to create quality prediction by direct association method using WHO dictonary

In [9]:
# Configutation part #

# Directory with nucleotide substitution files 
nucl_data_dir = '../db/nucl_data'
# Directory with phenotypes
pheno_dir = '../../pheno'
# Directory with matrices without split to parts and without excluding rare mutations
data_dir = ''
# WHO catalogue
who_catalogue_file = "../db/AL123456_rev.gff"
# Annotation file
annotation_file = "../db/AL123456_rev.gff"
# Reference fasta file
reference_file = '../db/h37rv.fasta'

######################

In [77]:
import os

import math

import pandas as pd
import numpy as np

from multiprocessing import Pool

from copy import deepcopy

from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
from Bio.Data import CodonTable

from tqdm import tqdm

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score

In [79]:
drug_dict = {
    'AMI': 'Amikacin',
    'CAP': 'Capreomycin',
    'EMB': 'Ethambutol',
    'ETH': 'Ethionamide',
    'INH': 'Isoniazid',
    'KAN': 'Kanamycin',
    'MXF': 'Moxifloxacin',
    'PZA': 'Pyrazinamide',
    'RIF': 'Rifampicin',
    'STM': 'Streptomycin'
}

drugs = drug_dict.keys()

nucl_dict = {
    'C': 'G',
    'G': 'C',
    'A': 'T',
    'T': 'A'
}
# Annotation preparation
annotation = pd.read_csv(annotation_file, sep='\t', index_col=None, header=None)
annotation = annotation[annotation[2] == 'Gene']
annotation[8] = [x.split(';')[0].split()[1] for x in annotation[8].to_numpy()]
annotation = annotation.astype({3: int, 4: int})

annotation[3] = annotation[3] - 1
annotation[4] = annotation[4] - 1

# Who catalogue preparation
who_catalogue = pd.read_csv(who_catalogue_file, 
                            sep=';', index_col=None, header=None)
who_catalogue = who_catalogue[who_catalogue[51].isin(['1) Assoc w R'])]
nucl_who_catalogue = who_catalogue[~(who_catalogue.isna()[3])]
nucl_who_catalogue = nucl_who_catalogue.loc[1:]

nucl_who_catalogue = nucl_who_catalogue.astype({7: int, 8: int, 9: int, 10: int})
nucl_who_catalogue[7] += nucl_who_catalogue[9]
nucl_who_catalogue[8] += nucl_who_catalogue[10]
nucl_who_catalogue = nucl_who_catalogue[[0, 2, 3, 7, 8, 9, 11]]
nucl_who_catalogue.columns = ['drug', 'desc', 'pos', 'pres_who', 'abs_who', 'pres_r_who', 'ppv_who']
nucl_who_catalogue['|'] = '|'

aa_snps_who_catalogue = who_catalogue[(who_catalogue.isna()[3])]
aa_snps_who_catalogue[7] += aa_snps_who_catalogue[9]
aa_snps_who_catalogue[8] += aa_snps_who_catalogue[10]
aa_snps_who_catalogue = aa_snps_who_catalogue[[0, 2, 3, 7, 8, 9, 11]]
aa_snps_who_catalogue.columns = ['drug', 'desc', 'pos', 'pres_who', 
                                 'abs_who', 'pres_r_who','ppv_who']
aa_snps_who_catalogue['|'] = '|'

indel_who_catalogue = nucl_who_catalogue[np.array([len(x.split('_')) == 6 for x in nucl_who_catalogue['desc'].values])]
nucl_snps_who_catalogue = nucl_who_catalogue[[True if ('ins' not in x) & \
                                              ('del' not in x) else False \
                                              for x in nucl_who_catalogue['desc'].values]]

# Form list of nucleotide samples name
samples = os.listdir(nucl_data_dir)
samples.remove('samples_low_covered.list')
samples.remove('samples_filtered.list')
samples.remove('combined1.list')
samples.remove('combined2.list')
samples.remove('combined3.list')
samples.remove('combined4.list')
samples.remove('combined5.list')
samples.remove('filtered_raw_variants_pos.csv')
samples.remove('low_covered_raw_variants_pos.csv')

# Reference 
f = open('../db/h37rv.fasta', 'r')
reference = "".join([x[:-1] if x[-1] == '\n' else x for x in f.readlines()[1:]])
f.close()

comp_reference = "".join([nucl_dict[x] for x in reference][::-1])

LENGHT_GEN = len(reference)

/export/home/fedonin/reshetnikov/env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (1,3,4,5,6,7,8,9,10,25,28,31,34,36,40,41,42,43,44,45,46,47,48,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/export/home/fedonin/reshetnikov/env/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/export/home/fedonin/reshetnikov/env/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

## Prepare matrix containing indels information

In [12]:
def converter(mut, pos, reference):
    pos = LENGHT_GEN - pos + 1
    LEN_SEQ = 100

    e = 20
    # print(reference[pos-e:pos+3])
    # print("".join([nucl_dict[x] for x in reference[pos-e:pos+3]]))

    new_reference = deepcopy(reference)
    if mut.split('_')[2] == 'del':
        fr = mut.split('_')[4].upper()
        to = mut.split('_')[5].upper()
        
        fr_comp_rev = ''.join([nucl_dict[x] for x in list(fr)[::-1]])

        if fr_comp_rev[:-1] != reference[pos-len(fr):pos-1]:
            print("Error: complement substitution is not equal to the reference!")
            
        new_reference = reference[:pos-len(fr)] + reference[pos-1:]

        return new_reference[pos-len(fr) - LEN_SEQ:pos-len(fr) + LEN_SEQ], \
                reference[pos - LEN_SEQ*2:pos + LEN_SEQ*2], \
                pos - LEN_SEQ*2
    else:
        fr = mut.split('_')[4].upper()
        to = mut.split('_')[5].upper()
        
        fr_comp_rev = ''.join([nucl_dict[x] for x in list(fr)[::-1]])
        to_comp_rev = ''.join([nucl_dict[x] for x in list(to)[::-1]])

        # if nucl_dict[fr] != reference[pos-1]:
        #     print("Error: complement substitution is not equal to the reference!")
        new_reference = reference[:pos-len(fr)] + to_comp_rev + reference[pos:] 
        
        return new_reference[pos - LEN_SEQ:pos + LEN_SEQ], \
                reference[pos - LEN_SEQ*2:pos + LEN_SEQ*2], \
                pos - LEN_SEQ*2

def caller(alignment, shift):
    seq_1 = alignment[0][0]
    seq_2 = alignment[0][1]
    
    gap_st = -1
    gap_end = -1
    flag = 0
    for i, n in enumerate(seq_1):
        if (flag == 0) & (n != '-'):
            flag = 1
        if (flag == 1) & (n == '-'):
            gap_st = i
            flag = 2
        if (flag == 2) & (n != '-'):
            gap_end = i
            break
    
    if gap_end == -1:
        min_shift = LENGHT_GEN
        min_j = 0
        for j in range(0, len(alignment)):
            flag = 0
            for i, n in enumerate(alignment[j][1]):
                if (flag == 0) & (n != '-'):
                    flag = 1
                if (flag == 1) & (n == '-'):
                    if i < min_shift:
                        min_shift = i
                        gap_st = i
                        flag = 2
                if (flag == 2) & (n != '-'):
                    gap_end = i
                    min_j = j
                    break
        seq_1 = alignment[min_j][0]
        seq_2 = alignment[min_j][1] 

        return [gap_st-1+1+shift, seq_1[gap_st-1:gap_end], seq_1[gap_st-1]]
    else:
        min_shift = gap_st
        min_j = 0
        for j in range(1, len(alignment)):
            flag = 0
            for i, n in enumerate(alignment[j][0]):
                if (flag == 0) & (n != '-'):
                    flag = 1
                if (flag == 1) & (n == '-'):
                    if i < min_shift:
                        min_shift = i
                        gap_st = i
                        flag = 2
                if (flag == 2) & (n != '-'):
                    gap_end = i
                    min_j = j
                    break
        seq_1 = alignment[min_j][0]
        seq_2 = alignment[min_j][1] 
        
        return [gap_st-1+1+shift, seq_2[gap_st-1], seq_2[gap_st-1:gap_end]]

In [ ]:
list_mut_converted = []
for mut, pos, in tqdm(indel_who_catalogue[['desc','pos']].to_numpy()[:2]):
    pos = int(pos)
    changed_seq, seq, shift = converter(mut, pos, reference)
    alignment = pairwise2.align.globalxs(reference, seq,-1,0)
    list_mut_converted.append(caller(alignment, shift))
indel_who_catalogue['conv_mut'] = ["_".join([str(y) for y in x]) for x in list_mut_converted]
indel_who_catalogue.to_csv("indel_info.csv", sep='\t', index=True, header=True)

In [16]:
indel_who_catalogue = pd.read_csv("indel_info.csv", sep='\t', index_col=0, header=0)

In [30]:
output = dict.fromkeys(indel_who_catalogue['conv_mut'].values)

for mut in indel_who_catalogue['conv_mut'].values:
    output[mut] = []

list_mut_conv_mod = ["\t".join([str(y) for y in x.split('_')]) + '\n' for x in indel_who_catalogue['conv_mut'].values]
    
for sample in tqdm(samples):
    sample_name = sample.split('.')[0]
    f = open(f'{nucl_data_dir}/{sample}', 'r')
    for line in f.readlines():
        if line in list_mut_conv_mod:
            [pos, fr, to] = line.split('\t')
            to = to[:-1]
            target = '_'.join([pos, fr, to])
            output[target].append(sample_name)
    f.close()
    
filtered_indel_who_catalogue = indel_who_catalogue[indel_who_catalogue.index != 13044]

data = pd.DataFrame(0, index=[x.split('.')[0] for x in samples], 
                      columns=filtered_indel_who_catalogue['conv_mut'].values)
for mut in indel_who_catalogue['conv_mut'].values:
    data.loc[output[mut], mut] = 1
    
data.to_csv("indel_matrix.csv", sep='\t', index=True, header=True)


100%|██████████| 12814/12814 [00:19<00:00, 657.24it/s]


## Prepare matric containing nucleotide snps information

In [37]:
def return_strain(annotation, gene):
    return annotation[annotation[8] == gene][6].values[0]

def convert_who(mut, pos, annotation, reference):
    
    pos = LENGHT_GEN - int(pos) + 1
    
    if len(mut.split()) == 2:
        mut_1 = mut.split()[0]
        mut_2 = mut.split()[1][1:-1]
        
        pos_1 = int(mut_1.split('_')[1][1:-1])
        pos_2 = int(mut_2.split('_')[1][1:-1])
        
        if abs(pos_1) <= abs(pos_2):
            mut = mut_1
        else:
            mut = mut_2
            
    gene = mut.split('_')[0]
    
        
    if len(mut.split('_')) == 2:
        subst = mut.split('_')[1]
        
        if return_strain(annotation, gene) == '-':
            fr = nucl_dict[subst[0].upper()]
            to = nucl_dict[subst[-1].upper()]
        else:
            fr = subst[0].upper()
            to = subst[-1].upper()
        
        return [pos, fr, to]
    else:
        return None

In [40]:
list_of_snps_mut = [convert_who(mut, pos, annotation, reference) for mut, pos \
                    in nucl_snps_who_catalogue[['desc', 'pos']].to_numpy()]
list_of_snps_mut = ['_'.join([str(y) for y in x]) for x in list_of_snps_mut]

drug_list = [drug_dict[x] for x in nucl_snps_who_catalogue['drug']]

nucl_snps_who_catalogue['raw_name'] = list_of_snps_mut

nucl_snps_who_catalogue.to_csv("nucl_snps_info.csv", sep='\t', index=False, header=True)

/export/home/fedonin/reshetnikov/env/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [41]:
output = dict.fromkeys(list_of_snps_mut, '')

for sample in tqdm(samples):
    sample_name = sample.split('.')[0]
    
    f = open(f"{nucl_data_dir}/{sample}", 'r')
    for mut in f.readlines():
        mut = "_".join(mut[:-1].split('\t'))
        
        if mut in list_of_snps_mut:
            if output[mut] == '':
                output[mut] = [sample_name]
            else:
                output[mut].append(sample_name)
    f.close()
    
data = pd.DataFrame(0, index=[x.split('.')[0] for x in samples], columns=np.unique(list_of_snps_mut))

for mut in output.keys():
    indel_data.loc[output[mut], mut] = 1
    
data.to_csv("nucl_snps_matrix.csv", sep='\t', index=True, header=True)

100%|██████████| 12814/12814 [00:27<00:00, 470.64it/s]


## Prepare matric containing aminoacids snps information

In [47]:
aa_snps_who_catalogue.to_csv("aa_snps_info.csv", sep='\t', index=False, header=True)

In [85]:
gene_list

array(['embB', 'ethA', 'gid', 'gyrA', 'gyrB', 'katG', 'pncA', 'rplC',
       'rpoB', 'rpsL', 'tlyA'], dtype='<U4')

In [51]:
gene_list = np.unique([x.split('_')[0] for x in aa_snps_who_catalogue['desc']])
anno_gene_list = annotation[[8, 3, 4, 6]][annotation[8].isin(gene_list)].to_numpy()

In [56]:
result = dict.fromkeys([x.split('.')[0] for x in samples])
all_features = set()

for sample in tqdm(samples):
    sample_name = sample.split('.')[0]
    output = []
    f = open(f'{nucl_data_dir}/{sample}', 'r')
    for line in f.readlines():
        if len(line.split('\t')) == 1:
            print(sample)
        pos, fr, to = line.split('\t')

        pos = int(pos) - 1
        to = to[:-1]

        for [gene, st, end, strand] in anno_gene_list:

            if (st <= pos) & (pos < end) & (len(fr) == 1) & (len(to) == 1):

                gene_seq = reference[st:end+1]
                gene_pos = pos - st

                codon_number = math.floor(gene_pos / 3)
                codon_pos = gene_pos % 3

                codon_seq = gene_seq[(codon_number) * 3:(codon_number + 1) * 3]

                if codon_seq[codon_pos] != fr:
                    print('Error! Substitution is not equal to the reference')

                if codon_pos == 0:
                    mod_codon_seq = to + codon_seq[codon_pos+1:]
                elif codon_pos == 1:
                    mod_codon_seq = codon_seq[:codon_pos]+ to + codon_seq[codon_pos+1:]
                else:
                    mod_codon_seq = codon_seq[:codon_pos] + to

                if strand == '-':
                    codon_number = int(len(gene_seq)/3) - (codon_number + 1)

                    codon_A = Seq(codon_seq).reverse_complement().translate(stop_symbol='!')

                    codon_B = Seq(mod_codon_seq).reverse_complement().translate(stop_symbol='!')
                else:
                    codon_A = Seq(codon_seq).translate(stop_symbol='!')

                    codon_B = Seq(mod_codon_seq).translate(stop_symbol='!')

                if codon_A == codon_B:
                    continue

                output.append(f'{gene}_{codon_A}{codon_number+1}{codon_B}')
    f.close()
    
    all_features = all_features | set(output)
    result[sample_name] = output
    
final = pd.DataFrame(0, columns=list(all_features), index=[x.split('.')[0] for x in samples])

for sample in tqdm(samples):
    sample_name = sample.split('.')[0]
    final.loc[sample_name, result[sample_name]] = 1
    
final.to_csv("aa_snps_matrix.csv", sep='\t', index=True, header=True)

100%|██████████| 12814/12814 [00:06<00:00, 2003.93it/s]


## Make prediction

In [80]:
def get_auc(y_test, y_proba):

    auc = 0
    try:
        auc = roc_auc_score(y_test, y_proba)
    except ValueError:
        pass
    return auc

In [81]:
indel_data = pd.read_csv("indel_matrix.csv", sep='\t', index_col=0, header=0)
nucl_snps_data = pd.read_csv("nucl_snps_matrix.csv", sep='\t', index_col=0, header=0)
aa_data = pd.read_csv('aa_snps_matrix.csv', sep='\t', index_col=0, header=0)

indel_info = pd.read_csv("indel_info.csv", sep='\t', index_col=None, header=0)
nucl_snps_info = pd.read_csv("nucl_snps_info.csv", sep='\t', index_col=None, header=0)
aminoacids_info = pd.read_csv("aa_snps_info.csv", sep='\t', index_col=None, header=0)

data = pd.concat([indel_data, nucl_snps_data, aa_data], axis=1)

In [88]:
auc = []
f1 = []
sensitivity = []
specificity = []
PPV = []
NPV = []

result = pd.DataFrame(index=[drug_dict[x] for x in drug_dict.keys()])

for drug in drugs:
    drug_name = drug_dict[drug]
    
    features = []
    
    features.extend(indel_info[indel_info['drug'] == drug]['conv_mut'].values)
    features.extend(nucl_snps_info[nucl_snps_info['drug'] == drug]['raw_name'].values)
    features.extend(aminoacids_info[aminoacids_info['drug'] == drug]['desc'].values)
    
    pheno = pd.read_csv(f"{pheno_dir}/{drug_name}.pheno", sep='\t', 
                        index_col=0, header=None, names=['pheno'])
    
    features = list(set(features) & set(data.columns))
    pheno_data = data[features]
    
    pheno_data['sum'] = np.sum(pheno_data, axis=1)
    pheno_data['sum'][pheno_data['sum'] != 0] = 1
    
    pheno_data = pd.merge(pheno_data['sum'], pheno, left_index=True, right_index=True, how='right')
    
    auc.append(get_auc(pheno_data['pheno'].values, pheno_data['sum'].values))
    f1.append(f1_score(pheno_data['pheno'].values, pheno_data['sum'].values))
    
    TP = np.sum(np.sum(pheno_data[pheno_data['sum'] == 1], axis=1) == 2)
    FP = np.sum(np.sum(pheno_data[pheno_data['sum'] == 1], axis=1) == 1)
    TN = np.sum(np.sum(pheno_data[pheno_data['sum'] == 0], axis=1) == 0)
    FN = np.sum(np.sum(pheno_data[pheno_data['sum'] == 0], axis=1) == 1)
    
    sensitivity.append(TP/(TP + FN))
    specificity.append(TN/(TN + FP))
    PPV.append(TP/(TP + FP))
    NPV.append(TN/(TN + FN))
    
result['auc'] = auc
result['sensitivity'] = sensitivity
result['specificity'] = specificity
result['PPV'] = PPV
result['NPV'] = NPV

result.to_csv('TableS23', sep='\t', index=True, header=True)

/export/home/fedonin/reshetnikov/env/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/export/home/fedonin/reshetnikov/env/lib/python3.6/site-packages/pandas/core/series.py:1021: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)
/export/home/fedonin/reshetnikov/env/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 